In [10]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [1]:
import re
from bs4 import BeautifulSoup
import re, string, unicodedata

def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)


def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()


def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words.split():
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word.replace('\n', ' '))
    return ' '.join(new_words)

In [3]:
import json
from pprint import pprint

everything = []
data = json.load(open('Archive/ComedyCemetery.json'))
data2 = json.load(open('Archive/dankmemes.json'))
data3 = json.load(open('Archive/MemeEconomy.json'))
data4 = json.load(open('Archive/memes.json'))
data5 = json.load(open('Archive/wholesomememes.json'))

for meme in data:
    title = remove_emoji(meme["title"])
    everything.append(remove_non_ascii(title))

for meme in data2:
    title = remove_emoji(meme["title"])
    everything.append(remove_non_ascii(title))

for meme in data3:
    title = remove_emoji(meme["title"])
    everything.append(remove_non_ascii(title))

for meme in data4:
    title = remove_emoji(meme["title"])
    everything.append(remove_non_ascii(title))

for meme in data5:
    title = remove_emoji(meme["title"])
    everything.append(remove_non_ascii(title))

In [6]:
with open('meme_captions.txt', 'w') as f:
    for e in everything:
        f.write(e)
        f.write('\n')

In [20]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

def save_doc(lines, filename):
    data = '\n'.join(lines)
    file = open(filename, 'w')
    file.write(data)
    file.close()

In [11]:
'''
cleaning and preprocessing of the meme captions. 
prepend each with <s> and append with </s>. --> TO DO take this out?
in separate file also working to just do padding of single captions rather than combining all with <s> and </s> delimiters
^ I'll push this one to the repo later, have to finish it first :) it will probably work better though 
'''

import string

filename = 'meme_captions.txt'

# load captions into memory
lines = open(filename).read().splitlines()

# tokenize and preprocess each caption
all_tokens = []
for caption in lines:
    # split into tokens by white space
    tokens = caption.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    tokens.insert(0, '<s>')
    tokens.append('</s>')
    all_tokens.extend(tokens)

# organize into sequences of 20 tokens
length = 20 + 1
sequences = list()
for i in range(length, len(all_tokens)):
    # select sequence of tokens
    seq = all_tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

    
# save sequences to file
out_filename = 'cleaned_captions.txt'
save_doc(sequences, out_filename)

Total Sequences: 193001


In [12]:
lines = open('cleaned_captions.txt').read().splitlines()

# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [13]:
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 50)            707900    
_________________________________________________________________
lstm_1 (LSTM)                (None, 20, 100)           60400     
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 14158)             1429958   
Total params: 2,288,758
Trainable params: 2,288,758
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=50)

Epoch 1/50
193001/193001 [==============================] - 1729s 9ms/step - loss: 3.6162 - acc: 0.3000
Epoch 2/50
193001/193001 [==============================] - 1229s 6ms/step - loss: 3.5652 - acc: 0.3052
Epoch 3/50
193001/193001 [==============================] - 1292s 7ms/step - loss: 3.5108 - acc: 0.3112
Epoch 4/50
193001/193001 [==============================] - 1520s 8ms/step - loss: 3.5378 - acc: 0.3100
Epoch 5/50
193001/193001 [==============================] - 1289s 7ms/step - loss: 3.6040 - acc: 0.3063
Epoch 6/50
193001/193001 [==============================] - 1254s 6ms/step - loss: 3.6094 - acc: 0.3074
Epoch 7/50
193001/193001 [==============================] - 1325s 7ms/step - loss: 3.5144 - acc: 0.3143
Epoch 8/50
 71808/193001 [==========>...................] - ETA: 15:00 - loss: 3.4167 - acc: 0.3253

In [34]:
# save the model to file
model.save('caption_model.h5')
# save the tokenizer
dump(tokenizer, open('caption_tokenizer.pkl', 'wb'))

In [32]:
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

# load the model
model = load_model('caption_model.h5')
tokenizer = load(open('caption_tokenizer.pkl', 'rb'))

# load cleaned text sequences
in_filename = 'cleaned_captions.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1

seed_text = lines[randint(0,len(lines))]
print('seed text: \n')
print(seed_text + '\n')

n_words = 20

result = list()
in_text = seed_text
# generate a fixed number of words
for _ in range(n_words):
    # encode the text as integer
    encoded = tokenizer.texts_to_sequences([in_text])[0]
    # truncate sequences to a fixed length
    encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
    # predict probabilities for each word
    yhat = model.predict_classes(encoded, verbose=0)
    # map predicted word index to word
    out_word = ''
    for word, index in tokenizer.word_index.items():
        if index == yhat:
            out_word = word
            break
    # append to input
    in_text += ' ' + out_word
    result.append(out_word)
    
meme_caption = ' '.join(result)
print('generated caption: \n')
print(meme_caption)

always wanted to contribute i hope this drawing i did is wholesome enough </s> <s> this is fantastic have some karma

server s s i guess i was a good friend s s i love this thread s s i love
